In [1]:
from socceraction.spadl.statsbomb import convert_to_actions
from socceraction.data.statsbomb import StatsBombLoader
from socceraction.spadl.config import actiontypes, bodyparts
import socceraction.spadl as spadl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, mean_squared_error
from xgboost import XGBClassifier, XGBRegressor
import math
import pickle
from mplsoccer import Pitch
import hashlib
import copy
import random

In [2]:
api = StatsBombLoader(root="data/statsbomb", getter="local")

In [3]:
# Competitions Input
competitions_df = api.competitions()
competitions_df[competitions_df["country_name"] == "International"]

,season_id,competition_id,competition_name,country_name,competition_gender,season_name
25,274,1470,FIFA U20 World Cup,International,male,1979
26,106,43,FIFA World Cup,International,male,2022
27,3,43,FIFA World Cup,International,male,2018
28,55,43,FIFA World Cup,International,male,1990
29,54,43,FIFA World Cup,International,male,1986
30,51,43,FIFA World Cup,International,male,1974
31,272,43,FIFA World Cup,International,male,1970
32,270,43,FIFA World Cup,International,male,1962
33,269,43,FIFA World Cup,International,male,1958
68,107,72,Women's World Cup,International,female,2023


In [4]:
# Games Input (Competition_id, Season_id)
games_world_cup_2018_df = api.games(43, 106)
games_world_cup_2018_df[games_world_cup_2018_df['game_id'] == 3857292] # Germany vs Costa Rica

,game_id,season_id,competition_id,competition_stage,game_day,game_date,home_team_id,away_team_id,home_score,away_score,venue,referee
43,3857292,106,43,Group Stage,3,2022-12-01 21:00:00,795,770,2,4,Al Bayt Stadium,Stéphanie Frappart


In [5]:
# Teams Input (game_id)
teams_world_cup_2018_df = api.teams(3857292)
teams_world_cup_2018_df.head()

,team_id,team_name
0,770,Germany
1,795,Costa Rica


In [6]:
# Player Input (game_id)
players_df = api.players(3857292)
players_df

,game_id,team_id,player_id,player_name,nickname,jersey_number,is_starter,starting_position_id,starting_position_name,minutes_played
0,3857292,770,3053,Leroy Sané,None,19,True,17,Right Wing,101
1,3857292,770,3167,Antonio Rüdiger,None,2,True,5,Left Center Back,101
2,3857292,770,5562,Thomas Müller,None,13,True,23,Center Forward,66
3,3857292,770,5570,Manuel Neuer,None,1,True,1,Goalkeeper,101
4,3857292,770,5579,Joshua Kimmich,None,6,True,2,Right Back,101
5,3857292,770,6322,Niklas Süle,None,15,True,3,Right Center Back,94
6,3857292,770,6324,Leon Goretzka,None,8,True,9,Right Defensive Midfield,45
7,3857292,770,8227,Mario Götze,None,11,False,0,Substitute,35
8,3857292,770,8400,Serge Gnabry,None,10,True,21,Left Wing,101
9,3857292,770,8507,Lukas Klostermann,None,16,False,0,Substitute,56


In [7]:
# # Load 360 data
# list_competitions_ids = []
# list_game_ids = []

# competitions_df = api.competitions()
# for _, row in competitions_df.iterrows():
#     if (row['competition_gender'] == 'male'):
#         list_competitions_ids.append((row['competition_id'], row['season_id']))

# for competition_id, season_id in list_competitions_ids:
#     games_df = api.games(competition_id, season_id)
#     for _, row in games_df.iterrows():
#         list_game_ids.append((row['game_id'], row['home_team_id'], row['away_team_id']))

# for game_id, home_team_id, away_team_id in list_game_ids:
#     try:
#         this_game_events_df = api.events(game_id, load_360=True)
#         this_game_events_df = spadl.add_names(convert_to_actions(this_game_events_df, home_team_id))
#         this_game_events_df.to_csv(f'data/statsbomb_event_360_data/{game_id}_{home_team_id}_{away_team_id}_statsbomb_event_360.csv')
#     except FileNotFoundError:
#         print(f'File not found {game_id}-{home_team_id}-{away_team_id}')

game_id_test_360 = 3857292
home_team_id = 795
this_game_events_df = api.events(game_id_test_360, load_360=True)
this_game_events_df_spadl = spadl.add_names(convert_to_actions(this_game_events_df, home_team_id))
this_game_events_df_spadl_with_360 = pd.merge(this_game_events_df_spadl, this_game_events_df[["event_id", "visible_area_360", "freeze_frame_360"]], how="inner", left_on="original_event_id", right_on="event_id")
this_game_events_df_spadl_with_360

,game_id,original_event_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,type_id,result_id,bodypart_id,action_id,type_name,result_name,bodypart_name,event_id,visible_area_360,freeze_frame_360
0,3857292,89222d44-fc73-41a2-942f-dc837959873f,1,0.0,770,5562.0,52.058824,33.655696,70.500000,30.298734,0,1,5,0,pass,success,foot_right,89222d44-fc73-41a2-942f-dc837959873f,"[15.7598634398169, 80.0, 43.9327838131471, 3.4...","[{'teammate': True, 'actor': False, 'keeper': ..."
1,3857292,9eadc5d3-292a-462b-a1bd-a34fac5fb6a9,1,1.0,770,3167.0,70.500000,30.298734,70.764706,30.470886,21,1,0,1,dribble,success,foot,9eadc5d3-292a-462b-a1bd-a34fac5fb6a9,"[13.2790014582807, 80.0, 42.0790591327844, 1.7...","[{'teammate': True, 'actor': False, 'keeper': ..."
2,3857292,ba04d768-1ca4-4271-a1c3-4a8466e4081f,1,2.0,770,3167.0,70.764706,30.470886,70.058824,49.493671,0,1,5,2,pass,success,foot_right,ba04d768-1ca4-4271-a1c3-4a8466e4081f,"[13.022407186745, 80.0, 40.5890114411885, 3.42...","[{'teammate': True, 'actor': False, 'keeper': ..."
3,3857292,1f2f0b53-180e-4838-8c2b-3755c39b5928,1,4.0,770,5579.0,70.058824,49.493671,64.235294,52.678481,21,1,0,3,dribble,success,foot,1f2f0b53-180e-4838-8c2b-3755c39b5928,"[12.6945729391703, 80.0, 38.3645454457976, 9.3...","[{'teammate': True, 'actor': False, 'keeper': ..."
4,3857292,6e4dd6da-2414-4c83-96ac-627641a5604e,1,5.0,770,5579.0,64.235294,52.678481,26.558824,38.217722,0,0,5,4,pass,fail,foot_right,6e4dd6da-2414-4c83-96ac-627641a5604e,"[25.9736650031387, 80.0, 43.2857381397718, 14....","[{'teammate': True, 'actor': False, 'keeper': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,3857292,c543188e-a160-4a11-934a-852642134bb6,2,3267.0,795,5597.0,1.147059,34.688608,1.147059,34.688608,14,1,2,2293,keeper_save,success,other,c543188e-a160-4a11-934a-852642134bb6,"[0.0, 80.0, 0.0, 24.1379617609332, 12.74540221...","[{'teammate': True, 'actor': True, 'keeper': T..."
2237,3857292,39597c2c-4651-4959-b85d-7787bc489726,2,3272.0,795,5597.0,10.500000,28.318987,34.235294,47.686076,0,1,2,2295,pass,success,other,39597c2c-4651-4959-b85d-7787bc489726,NaN,NaN
2238,3857292,4a4a9fe3-9e86-402e-af88-e20723a1227a,2,3274.0,795,5581.0,34.235294,47.686076,31.852941,50.440506,21,1,0,2296,dribble,success,foot,4a4a9fe3-9e86-402e-af88-e20723a1227a,"[0.0, 80.0, 0.0, 63.6994672302392, 28.23736471...","[{'teammate': False, 'actor': False, 'keeper':..."
2239,3857292,22735c8a-7950-4cda-bf34-3c1fa20ee470,2,3279.0,770,8966.0,32.647059,49.665823,32.647059,49.665823,8,1,0,2297,foul,success,foot,22735c8a-7950-4cda-bf34-3c1fa20ee470,"[91.104079612647, 80.0, 77.1549282775432, 80.0...","[{'teammate': True, 'actor': False, 'keeper': ..."
